# Machine learning using pyspark on databricks
# Databricks
An open and unified data analytics platform for data engineering, data science, machine learning, and analytics using pyspark or apache spark where they can provide instances and clusters.

Interactive notebooks to use Apache SparkTM, SQL, Python, Scala, Delta Lake, MLflow, TensorFlow, Keras, Scikit-learn and more.
### Step1: go to databricks website 
![](3.png)
### Step2: create a cluster 
![](4.png)
![](5.png)
![](6.png)
### Step3: upload the data 
![](ml1.png)

### Step4: Create a Notebook and write the pyspark job

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

df=spark.read.csv(file_location,header=True,inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[54]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
#handling categorical features
from pyspark.ml.feature import StringIndexer  #this string indexer will convert the string categorical features in numerical features
indexer=StringIndexer(inputCols=["sex","smoker","day","time"],outputCols=["{}_indexed".format(c) for c in ['sex','smoker','day','time']])

In [0]:
df1=indexer.fit(df).transform(df)

In [0]:
df1.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
final_df=df1.drop("sex","smoker","day","time")

In [0]:
final_df.show()

+----------+----+----+-----------+--------------+-----------+------------+
|total_bill| tip|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+----+-----------+--------------+-----------+------------+
|     16.99|1.01|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|   2|        0.0|           0.0|        1.0|         0.0|
|     26.88|3.12|   4|        0.0|           0.0|        1.0|         0.0|
|     15.04|1.96|   2|        0.0|           0.0|        1.0|         0.0|
|     14.78|3.23|   2|        0.0|           0.0|        1.0|         0.0|
|     10.27|1.71|   2|   

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'],outputCol="independent_feature")
output=featureassembler.transform(final_df)

In [0]:
output.show()

+----------+----+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|size|sex_indexed|smoker_indexed|day_indexed|time_indexed| independent_feature|
+----------+----+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|   4|        0.0|           0.0|        1.0|         0.0|[4.71,4.0,0.0,0.0...|
|      8.77| 2.0|   2|        0.0|           0.0|        1.0|         0.0|[2.0,2.0,0.0,0.0,...|
|     26.88|3.12|   4|        0.0|      

In [0]:
finalized_data=output.select("independent_feature","total_bill")

In [0]:
finalized_data.show()

+--------------------+----------+
| independent_feature|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="independent_feature",labelCol="total_bill")
regressor=regressor.fit(train_data)

In [0]:
prediction=regressor.evaluate(test_data)

In [0]:
prediction.predictions.show()

+--------------------+----------+------------------+
| independent_feature|total_bill|        prediction|
+--------------------+----------+------------------+
| (6,[0,1],[2.0,2.0])|     13.37|14.636410915312766|
| (6,[0,1],[2.0,3.0])|     16.31|17.555595673932856|
|(6,[0,1],[2.01,2.0])|     20.23|14.668416236408655|
|(6,[0,1],[2.31,3.0])|     18.69|18.547760627905465|
|(6,[0,1],[3.27,2.0])|     17.78| 18.70108669449088|
|(6,[0,1],[4.08,2.0])|     17.92| 21.29351770325802|
| (6,[0,1],[5.0,3.0])|     31.27|27.157192002700047|
|(6,[0,1],[6.73,4.0])|     48.27| 35.61329731090921|
|[1.25,2.0,1.0,0.0...|      8.51| 9.836526903201731|
|[1.5,2.0,0.0,1.0,...|     11.59|15.844699412247008|
|[1.5,2.0,0.0,1.0,...|     12.03|16.328392384264333|
|[1.5,2.0,1.0,0.0,...|      8.35|10.636659930598997|
|[1.8,2.0,1.0,0.0,...|     12.43|11.596819563475718|
|[1.96,2.0,0.0,0.0...|     15.04|14.669620621601645|
|[1.98,2.0,0.0,1.0...|     11.02|17.380954824849763|
|[2.0,2.0,0.0,0.0,...|      8.77|14.7976419059

In [0]:
#performance matrix
prediction.r2,prediction.meanAbsoluteError,prediction.meanSquaredError

Out[82]: (0.5315638690964258, 4.312887887339609, 35.190133282188974)